In [1]:
import os
import pandas as pd
from pathlib import Path
from gns_utils import prepare_expr_files, csv_log_to_dataframe

EXPR_DIR = Path("gns_experiments")
CKPT_DIR = Path("checkpoints")
all_models = sorted(os.listdir(CKPT_DIR))
all_models

['0010000.pt', '0020000.pt', '0700000.pt', '0750000.pt']

In [2]:
def compare_estimation_parameters(
        name: str,
        expr_dir: str,
        models: [str],
        Bb_ratio: int,
        reps: int,
        min_size: int,
        max_size: int,
        b_step: int,
    ):
    assert len(models) > 1, "You must specify at least one model!"
    ## Process arguments and create .sh/.csv names
    shell_name, csv_name, vis_dir = prepare_expr_files(expr_name=name, expr_dir=expr_dir)

    ## Create experiment shells
    expr_count = 0
    for model in models:
        ## Check if model exists
        path = f"{CKPT_DIR}/{model}"
        assert os.path.exists(path), f"Checkpoint {path} does not exist!"

        ## Initialize set sizes (smallest)
        b = min_size
        B = Bb_ratio * b

        ## Iterate over batch size values
        while min_size <= b < B <= max_size:
            ## Create experiment with given values
            args = f"-b {b} -B {B} -r {reps}"
            create_experiment_bash_with(args, model, bash_path=shell_name, csv_path=csv_name, vis_dir=vis_dir)
            expr_count += 1
            ## Set new batch size values
            b += b_step
            B = Bb_ratio * b

    print(f"{expr_count} experiments.")
    print(f"Execute in the root-dir using GPUs:")
    print("------------------------------------------\n")

In [3]:
path = EXPR_DIR / "1_true_grad_accuracy/results/"
files = [path / file for file in os.listdir(path)]
df = pd.concat([csv_log_to_dataframe(file, raw=True) for file in files])

/Users/egecimsir/Desktop/GenAI Practical/Critical-Multitask-Batch-Sizes-in-Diffusion-Models/gns_utils.py:150: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["runtime"] = pd.to_datetime(df["runtime"])


In [4]:
df

,model,true_portion,diff_steps,B,b,reps,t_min,t_max,accumulate,epoch,...,ckpt_dir,vis_dir,save_fig,date,gns_est,g_norm,b_true,runtime,user,host
0,0010000.pt,0.5,1000,500,50,5,NaN,NaN,True,False,...,checkpoints,/home/c/cimsir/GenAI_Practical/gns_experiments...,NaN,2025-01-28 00:26:58,12.470157,1.033059,25000,00:06:17.515130,cimsir,basalt.cip.ifi.lmu.de
0,0010000.pt,0.1,1000,500,50,5,NaN,NaN,True,False,...,checkpoints,/home/c/cimsir/GenAI_Practical/gns_experiments...,NaN,2025-01-27 12:27:41,2.799724,"tensor(1.0477, device='cuda:0')",5000,00:02:32.160207,cimsir,basalt.cip.ifi.lmu.de
1,0010000.pt,0.1,1000,500,50,5,NaN,NaN,True,False,...,checkpoints,/home/c/cimsir/GenAI_Practical/gns_experiments...,NaN,2025-01-27 23:22:52,0.735751,1.0329235792160034,5000,00:01:56.101521,cimsir,basalt.cip.ifi.lmu.de
0,0010000.pt,0.2,1000,500,50,5,NaN,NaN,True,False,...,checkpoints,/home/c/cimsir/GenAI_Practical/gns_experiments...,NaN,2025-01-27 23:32:32,1.958498,1.032797,10000,00:03:02.227157,cimsir,basalt.cip.ifi.lmu.de
